# Assistants API - 파일 검색

- 파일 검색은 독점 제품 정보나 사용자가 제공한 문서 등 모델 외부의 지식으로 도우미를 강화합니다. OpenAI는 자동으로 문서를 구문 분석 및 청크하고, 임베딩을 생성 및 저장하며, 벡터 및 키워드 검색을 모두 사용하여 관련 콘텐츠를 검색하여 사용자 쿼리에 응답합니다.  

- OpenAI의 Assistant에서 사용하는 파일 검색 기능은 RAG (Retrieval-Augmented Generation) 개념을 구현한 것입니다.

- 작동 방식
    - 문서 구문 분석 및 청크 처리: 파일을 작은 청크로 나누어 각 청크에 대한 임베딩을 생성합니다.
    - 임베딩 생성 및 벡터 저장: 각 청크를 벡터화하여 검색을 위한 벡터 데이터베이스에 저장합니다.
    - 벡터 및 키워드 검색: 사용자 쿼리와 관련된 벡터를 검색하거나 키워드 기반으로 관련 청크를 찾습니다.
    - 응답 생성: 검색된 청크를 바탕으로 사용자 질문에 대한 응답을 생성합니다.
  
이 과정에서 벡터 검색을 통해 관련 정보를 효과적으로 검색하고, 검색된 정보를 생성 과정에 결합하여 응답하므로, 이는 전형적인 RAG 아키텍처의 특징을 따릅니다.

이 실습에서는 회사의 재무제표에 대한 질문에 답변하는 데 도움이 되는  assistant를 만듭니다.

In [1]:
import os
import openai
import sys
sys.path.append('./')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

### 1단계: 파일 검색이 활성화된 새 assistant 만들기
Assistants tools parameter수에서 file_search가 활성화된 새 assistant를 만듭니다  
file_search 도구가 활성화되면 모델은 사용자 메시지를 기반으로 콘텐츠를 검색할 시기를 결정합니다.

In [2]:
from openai import OpenAI
client = OpenAI()

Model = "gpt-4o-mini"

In [3]:
assistant = client.beta.assistants.create(
  name="Financial Analyst Assistant",
  instructions="당신은 전문 재무 분석가입니다. 지식 기반을 사용하여 감사된 재무제표에 대한 질문에 답하십시오.",
  model=Model,
  tools=[{"type": "file_search"}],
)

assistant

Assistant(id='asst_MafHx9DKasI7OQlTZYtHXiUv', created_at=1731191986, description=None, instructions='당신은 전문 재무 분석가입니다. 지식 기반을 사용하여 감사된 재무제표에 대한 질문에 답하십시오.', metadata={}, model='gpt-4o-mini', name='Financial Analyst Assistant', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=[])), top_p=1.0)

### 2단계: Vector Store 생성

- 가격: FILE SEARCH 기능은 벡터 저장소(storage) 용량을 기준으로 하루에 $0.10/GB의 요금이 부과됩니다. 첫 번째 1GB는 무료로 제공됩니다​.
- 보존기간: 생성된 벡터 저장소가 일주일 동안 비활성 상태로 남아 있으면 자동으로 삭제됩니다.

In [4]:
# "Financial Statements"라는 벡터 스토어 생성
vector_store = client.beta.vector_stores.create(name="Financial Statements")
 
# OpenAI에 업로드할 파일 준비
file_paths = ["재무제표/goog-10k.pdf", "재무제표/brka-10k.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
 
# 업로드 및 폴링 SDK 도우미를 사용하여 파일을 업로드하고 벡터 스토어에 추가,
# 파일 배치의 완료 상태를 폴링합니다.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# 이 작업의 결과를 보기 위해 상태 및 파일 개수를 출력할 수 있습니다.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


### 3단계: 새로운 Vector Store를 사용하도록 어시스턴트 업데이트
어시스턴트가 파일에 액세스할 수 있도록 하려면 어시스턴트의 tool_resources를 새 vector_store ID로 업데이트합니다.

In [5]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)
assistant

Assistant(id='asst_MafHx9DKasI7OQlTZYtHXiUv', created_at=1731191986, description=None, instructions='당신은 전문 재무 분석가입니다. 지식 기반을 사용하여 감사된 재무제표에 대한 질문에 답하십시오.', metadata={}, model='gpt-4o-mini', name='Financial Analyst Assistant', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_MyioCf81vRYv2MgFMiAAbNdv'])), top_p=1.0)

### 4단계: 스레드 만들기
스레드에 메시지 첨부 파일로 파일을 첨부할 수도 있습니다. 이렇게 하면 스레드와 연결된 또 다른 벡터 저장소가 생성됩니다. 또는 이 스레드에 이미 연결된 벡터 저장소가 있는 경우 새 파일을 기존 스레드 벡터 저장소에 연결합니다. 이 스레드에서 실행을 생성하면 파일 검색 도구는 어시스턴트의 vector_store와 스레드의 vector_store를 모두 쿼리합니다.  

여기서 사용자는 Apple의 최신 10-K 파일을 추가로 첨부합니다. 

In [7]:
# OpenAI에 사용자 제공 파일 업로드
message_file = client.files.create(
  file=open("재무제표/aapl-10k.pdf", "rb"), purpose="assistants"
)

# 스레드 생성 및 파일을 메시지에 첨부
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "2023년 10월 말에 AAPL의 발행 주식 수는 얼마였나요?",
      # 새 파일을 메시지에 첨부합니다.
      "attachments": [
        { "file_id": message_file.id, 
          "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)

# 이제 스레드에는 파일이 있는 벡터 저장소가 툴 리소스로 있습니다.
print(thread.tool_resources.file_search)

ToolResourcesFileSearch(vector_store_ids=['vs_UVt4YugzqozQSoeDcfwuX902'])


### 5단계: Run 만들기 및 출력 확인
이제Run행을 생성하고 모델이 파일 검색 도구를 사용하여 사용자의 질문에 대한 응답을 제공하는 것을 관찰합니다.

In [8]:
# `create_and_poll` 메서드를 사용하여 새로운 실행(run)을 생성하고 해당 실행이 완료될 때까지 폴링합니다.
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,  # 실행할 스레드의 ID
  assistant_id=assistant.id,  # 실행할 어시스턴트의 ID
  instructions="사용자를 고객님이라고 부르세요. 이 사용자는 프리미엄 계정을 가지고 있습니다.",   # 어시스턴트에게 제공할 지시사항
)

In [9]:
# 실행 상태 확인
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_wAwadKaSf5L5A4daTJYXqyic', assistant_id='asst_MafHx9DKasI7OQlTZYtHXiUv', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=78, file_citation=FileCitation(file_id='file-bo8PksVakjNwOF9PoUWtglVd'), start_index=65, text='【4:15†source】', type='file_citation')], value='2023년 10월 말 기준으로 AAPL(Apple Inc.)의 발행 주식 수는 약 15,943,425,000 주입니다【4:15†source】.'), type='text')], created_at=1731192035, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_quXZSDSegGP2pNpN7GwyT7Rv', status=None, thread_id='thread_ogB2Rzu4eGwkIUOgAlm17axZ'), Message(id='msg_SRJl3L8HjHJVlYcgn3iyg7Yk', assistant_id=None, attachments=[Attachment(file_id='file-bo8PksVakjNwOF9PoUWtglVd', tools=[AttachmentToolAssistantToolsFileSearchTypeOnly(type='file_search')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='2023년 10월 말에 AAPL의 

In [10]:
messages.data[0].content[0].text.value

'2023년 10월 말 기준으로 AAPL(Apple Inc.)의 발행 주식 수는 약 15,943,425,000 주입니다【4:15†source】.'

### Thread에 새로운 message 추가 및 Run 생성

In [11]:
message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role="user",
      content="2023년 Google의 당기 순이익은 얼마였나요? BERKSHIRE HATHAWAY INC. 와 어느 쪽의 당기 순이익이 더 많았나요?"
    )

In [12]:
# `create_and_poll` 메서드를 사용하여 새로운 실행(run)을 생성하고 해당 실행이 완료될 때까지 폴링합니다.
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,  # 실행할 스레드의 ID
  assistant_id=assistant.id,  # 실행할 어시스턴트의 ID
  instructions="사용자를 고객님이라고 부르세요. 이 사용자는 프리미엄 계정을 가지고 있습니다.",   # 어시스턴트에게 제공할 지시사항
)

In [13]:
# 실행 상태 확인
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_PAtmmCG69GKfjgQ9puB1YhFP', assistant_id='asst_MafHx9DKasI7OQlTZYtHXiUv', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=78, file_citation=FileCitation(file_id='file-Ri5p1QZfQsOCqtxQkslqRYnS'), start_index=65, text='【8:18†source】', type='file_citation'), FileCitationAnnotation(end_index=163, file_citation=FileCitation(file_id='file-1UEL427LyZmLrm8y2UKKHWA8'), start_index=150, text='【8:16†source】', type='file_citation')], value='2023년 Google(Alphabet Inc.)의 당기 순이익은 약 60억 달러(60 billion USD)였습니다【8:18†source】. \n\n반면, Berkshire Hathaway Inc.의 당기 순이익은 약 425억 달러(42.5 billion USD)였습니다【8:16†source】. \n\n따라서, 2023년의 당기 순이익은 Berkshire Hathaway가 Google보다 더 많았습니다.'), type='text')], created_at=1731192046, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_aT0vgDSV513El1sqNgLtC8qK', status=None, thread_id='thread_ogB2Rzu4eGwk

In [14]:
messages.data[0].content[0].text.value

'2023년 Google(Alphabet Inc.)의 당기 순이익은 약 60억 달러(60 billion USD)였습니다【8:18†source】. \n\n반면, Berkshire Hathaway Inc.의 당기 순이익은 약 425억 달러(42.5 billion USD)였습니다【8:16†source】. \n\n따라서, 2023년의 당기 순이익은 Berkshire Hathaway가 Google보다 더 많았습니다.'

In [15]:
# messages 객체의 데이터에서 role과 content 값을 추출하여 리스트에 저장
values = [(messages.data[i].role, messages.data[i].content[0].text.value) for i in range(len(messages.data)-1, -1, -1)]

# 추출한 값들을 출력
for role, value in values:
    print(f"role: {role}")  # 역할(role) 출력
    print(value)  # 내용(value) 출력
    print(150*"-")  # 구분선 출력

role: user
2023년 10월 말에 AAPL의 발행 주식 수는 얼마였나요?
------------------------------------------------------------------------------------------------------------------------------------------------------
role: assistant
2023년 10월 말 기준으로 AAPL(Apple Inc.)의 발행 주식 수는 약 15,943,425,000 주입니다【4:15†source】.
------------------------------------------------------------------------------------------------------------------------------------------------------
role: user
2023년 Google의 당기 순이익은 얼마였나요? BERKSHIRE HATHAWAY INC. 와 어느 쪽의 당기 순이익이 더 많았나요?
------------------------------------------------------------------------------------------------------------------------------------------------------
role: assistant
2023년 Google(Alphabet Inc.)의 당기 순이익은 약 60억 달러(60 billion USD)였습니다【8:18†source】. 

반면, Berkshire Hathaway Inc.의 당기 순이익은 약 425억 달러(42.5 billion USD)였습니다【8:16†source】. 

따라서, 2023년의 당기 순이익은 Berkshire Hathaway가 Google보다 더 많았습니다.
----------------------------------------------------------------------------

### 한글 PDF 분석

In [16]:
# step1) 새 assistant 생성
assistant = client.beta.assistants.create(
  name="Financial Analyst Assistant",
  instructions="당신은 기업분석 전문가입니다. 지식 기반을 사용하여 증권사 기업분석에 대한 질문에 답하십시오.",
  model=Model,
  tools=[{"type": "file_search"}],
)

In [17]:
# step2) 새 벡터 스토어 생성
vector_store = client.beta.vector_stores.create(name="기업분석보고서")
 
# OpenAI에 업로드할 파일 준비
file_paths = ["재무제표/네이버.pdf", "재무제표/LG엔솔.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
 
# 업로드 및 폴링 SDK 도우미를 사용하여 파일을 업로드하고 벡터 스토어에 추가,
# 파일 배치의 완료 상태를 폴링합니다.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# 이 작업의 결과를 보기 위해 상태 및 파일 개수를 출력할 수 있습니다.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


In [18]:
# step3) assistant update
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [32]:
# OpenAI에 사용자 제공 파일 업로드
message_file = client.files.create(
  file=open("재무제표/에코프로비엠.pdf", "rb"), purpose="assistants"
)

# step 4) 스레드 생성 및 파일을 메시지에 첨부
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "LG에너지솔루션의 2024년 하반기 전망을 요약해 주세요. 에코프로비엠은 매출액이 줄은 것에 비해 왜 영업이익이 더 크게 줄어들었나요?",
      # 새 파일을 메시지에 첨부합니다.
      "attachments": [
        { "file_id": message_file.id, 
          "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)

# 이제 스레드에는 파일이 있는 벡터 저장소가 툴 리소스로 있습니다.
print(thread.tool_resources.file_search)

ToolResourcesFileSearch(vector_store_ids=['vs_CXZJz6IC98yl9ZfMHNDstjMa'])


In [33]:
# step 5) 새로운 실행(run)을 생성하고 해당 실행이 완료될 때까지 폴링
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,  # 실행할 스레드의 ID
  assistant_id=assistant.id,  # 실행할 어시스턴트의 ID
  instructions="사용자를 고객님이라고 부르세요. 이 사용자는 프리미엄 계정을 가지고 있습니다.",   # 어시스턴트에게 제공할 지시사항
)

In [39]:
# 실행 상태 확인
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages.data[0].content[0].text.value)
else:
  print(run.status)

고객님께서 요청하신 2023년 4분기 네이버의 영업이익과 한화 투자 증권이 목표가를 상향 조정한 이유를 다음과 같이 요약해 드립니다.

### 2023년 4분기 네이버 영업이익
- 네이버의 2023년 4분기 영업이익은 **4,055억 원**이었으며, 이는 시장 기대치에 부합한 수치입니다【12:0†source】【12:3†source】.

### 한화 투자 증권의 목표가 상향 조정 이유
- 한화 투자 증권은 다음과 같은 이유로 네이버의 목표가를 상향 조정했습니다:
  1. **AI 기반 광고 및 커머스 성과**가 실적으로 나타나거나,
  2. **웹툰 엔터테인먼트 상장 모멘텀**이 부각될 경우 추가적인 기업 가치 상승이 기대되고 있습니다【12:0†source】【12:1†source】.
- 특히 2024년에는 웹툰 영업이익이 흑자로 돌아설 것으로 예상되며, 커머스 수수료 전환이 본격화되고 AI 매출이 기여할 것으로 보입니다【12:6†source】【12:1†source】.

추가적인 질문이 있으시면 언제든지 말씀해 주세요!


In [38]:
# thread에 새로운 message 추가
message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role="user",
      content="2023년 4분기의 네이버의 영업이익은 얼마였나요? 한화 투자 증권이 목표가를 상향 조정한 이유는 무엇인가요?"
    )

run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,  # 실행할 스레드의 ID
  assistant_id=assistant.id,  # 실행할 어시스턴트의 ID
  instructions="사용자를 고객님이라고 부르세요. 이 사용자는 프리미엄 계정을 가지고 있습니다.",   # 어시스턴트에게 제공할 지시사항
)

if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages.data[0].content[0].text.value)
else:
  print(run.status)

고객님께서 요청하신 2023년 4분기 네이버의 영업이익과 한화 투자 증권이 목표가를 상향 조정한 이유를 다음과 같이 요약해 드립니다.

### 2023년 4분기 네이버 영업이익
- 네이버의 2023년 4분기 영업이익은 **4,055억 원**이었으며, 이는 시장 기대치에 부합한 수치입니다【12:0†source】【12:3†source】.

### 한화 투자 증권의 목표가 상향 조정 이유
- 한화 투자 증권은 다음과 같은 이유로 네이버의 목표가를 상향 조정했습니다:
  1. **AI 기반 광고 및 커머스 성과**가 실적으로 나타나거나,
  2. **웹툰 엔터테인먼트 상장 모멘텀**이 부각될 경우 추가적인 기업 가치 상승이 기대되고 있습니다【12:0†source】【12:1†source】.
- 특히 2024년에는 웹툰 영업이익이 흑자로 돌아설 것으로 예상되며, 커머스 수수료 전환이 본격화되고 AI 매출이 기여할 것으로 보입니다【12:6†source】【12:1†source】.

추가적인 질문이 있으시면 언제든지 말씀해 주세요!


## 실습 : Prompt 수정하여 실행